imports

In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px

Set current directory

In [ ]:
cd <your_cd>

## Data loading

load populatiom

In [11]:
population_paths = [x for x in os.walk(os.path.join('data', 'population'))]

file_paths = []

for folder in population_paths:
    for file_name in folder[2]:
        if '.CSV' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data\\population\\IHME_GBD_2019_POP_2019_Y2020M10D15.CSV']

In [6]:
population = pd.read_csv('data\\population\\IHME_GBD_2019_POP_2019_Y2020M10D15.CSV')

In [8]:
population.location_id.nunique()

675

In [9]:
population.year_id.unique()

array([2019], dtype=int64)

In [10]:
population.shape

(95175, 14)

In [12]:
population

,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
0,1,Global,1,male,1,Under 5,2019,44,Population,1,Number,3.423987e+08,3.523971e+08,3.324841e+08
1,1,Global,1,male,2,Early Neonatal,2019,44,Population,1,Number,1.330245e+06,1.368039e+06,1.292459e+06
2,1,Global,1,male,3,Late Neonatal,2019,44,Population,1,Number,3.951181e+06,4.063467e+06,3.839020e+06
3,1,Global,1,male,4,Post Neonatal,2019,44,Population,1,Number,6.292360e+07,6.472045e+07,6.113298e+07
4,1,Global,1,male,5,1 to 4,2019,44,Population,1,Number,2.741937e+08,2.822446e+08,2.662198e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95170,44638,North America,2,female,164,Birth,2019,44,Population,1,Number,2.050261e+06,2.193948e+06,1.919386e+06
95171,44638,North America,3,both,164,Birth,2019,44,Population,1,Number,4.200444e+06,4.494820e+06,3.932317e+06
95172,44642,Nordic Region,1,male,164,Birth,2019,44,Population,1,Number,1.498598e+05,1.642737e+05,1.369929e+05
95173,44642,Nordic Region,2,female,164,Birth,2019,44,Population,1,Number,1.419441e+05,1.556075e+05,1.297482e+05


In [57]:
population_age_group_id_to_name = {
    k: v for k, v in population[['age_group_id', 'age_group_name']].values
}

load population_code_book

In [58]:
population_code_book = pd.read_csv(
    os.path.join('data', 'population', 'code_book.csv')
).iloc[1:, 1:]

population_code_book

,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
1,1,Global,1,male,1,Under 5,1950,44,Population,1,Number,NaN,NaN,NaN
2,4,"Southeast Asia, East Asia, and Oceania",2,female,2,Early Neonatal,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,East Asia,3,both,3,Late Neonatal,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,China,NaN,NaN,4,Post Neonatal,1953,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,Democratic People's Republic of Korea,NaN,NaN,5,1 to 4,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,44788,Torbay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,44789,"Bristol, City of",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,44790,Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,44791,Cornwall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


load life expectancy

In [175]:
paths = [x for x in os.walk(os.path.join('data', 'life_expectancy'))]

file_paths = []

for folder in paths:
    for file_name in folder[2]:
        if '.csv' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data\\life_expectancy\\IHME-GBD_2019_DATA-2eb8428f-1.csv']

In [176]:
life_expectancy = pd.concat([pd.read_csv(x) for x in file_paths], axis=0)

In [178]:
life_expectancy.columns

Index(['measure_id', 'measure_name', 'location_id', 'location_name', 'sex_id',
       'sex_name', 'age_id', 'age_name', 'metric_id', 'metric_name', 'year',
       'val', 'upper', 'lower'],
      dtype='object')

In [196]:
life_expectancy_age_id_to_name = {str(k): v for k, v in life_expectancy[['age_id', 'age_name']].values}

In [197]:
life_expectancy_age_id_to_name

{'45': '105-109 years',
 '148': '110+ years',
 '5': '1-4 years',
 '6': '5-9 years',
 '7': '10-14 years',
 '8': '15-19 years',
 '9': '20-24 years',
 '10': '25-29 years',
 '11': '30-34 years',
 '12': '35-39 years',
 '13': '40-44 years',
 '14': '45-49 years',
 '15': '50-54 years',
 '16': '55-59 years',
 '17': '60-64 years',
 '18': '65-69 years',
 '19': '70-74 years',
 '20': '75-79 years',
 '28': '<1 year',
 '30': '80-84',
 '31': '85-89',
 '32': '90-94',
 '33': '95-99 years',
 '44': '100-104 years'}

In [152]:
for column in life_expectancy.columns:
    print(f'{column}:')
    display(life_expectancy[column].unique())

measure:


array([26], dtype=int64)

location:


array([ 10,  11, 210, 211, 106, 107,  12,  18,  19, 212, 218,  20,  26,
       108, 114, 115,  27,  28, 305, 320,  34,  35,  36, 116, 122,  43,
       349, 374, 123, 129, 130,  44,  50,  51, 376, 380, 422,  52,  58,
       131,  59,  60, 139,  66,  67,  68,  74,  75, 145, 146, 147,  76,
        82,  83,  84,  90, 153, 154,  91,  92,  98, 214, 215, 110, 111,
         6,   7,   8, 155, 161, 162,  99, 216,  14,  15, 112, 118, 119,
       163, 169, 367, 393, 126,  16,  22,  23, 170, 171, 396, 413, 127,
       133,  24,  30, 135, 177, 178, 179, 141, 142, 143, 185, 186, 149,
       150, 151, 187, 194, 195,  38,  39,  40, 157, 196, 202,  46,  47,
       165, 173, 203, 204,  48,  54,  55, 175,  62, 181, 182,  63, 183,
       190, 191,  71,  72, 198,  78,  79, 200, 206,  80,  86,  87, 207,
       208,  88,  94,  95, 102, 213, 109, 117,  13, 351, 125, 132, 385,
        29, 435, 105, 113, 217, 140,  37,  17, 121, 298, 148,  45, 156,
        25, 369, 128,  53, 164, 136, 416,  61, 172,  33,  69, 18

sex:


...

array([ 1.68197025,  1.86740483,  1.3646299 , ..., 42.62414089,
       37.23417352, 38.20404758])

upper:


array([ 1.88519795,  2.13847693,  1.49018206, ..., 44.46501424,
       39.10290899, 39.93682735])

lower:


array([ 1.60690099,  1.71638231,  1.31755456, ..., 40.25761402,
       34.81024544, 35.88694192])

load life expectancy code book

In [153]:
life_expectancy_code_book = pd.read_csv(
    os.path.join('data', 'life_expectancy', 'code_book.csv')
).iloc[1:, 1:]

life_expectancy_code_book

,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
1,1,Global,1,male,5,1 to 4,1950,26,Life expectancy,5,Years,NaN,NaN,NaN
2,4,"Southeast Asia, East Asia, and Oceania",2,female,6,5 to 9,1951,27,Probability of death,8,Probability of death,NaN,NaN,NaN
3,5,East Asia,3,both,7,10 to 14,1952,31,Life expectancy no-shock with hiv,NaN,NaN,NaN,NaN,NaN
4,6,China,NaN,NaN,8,15 to 19,1953,33,Probability of death no-shock with hiv,NaN,NaN,NaN,NaN,NaN
5,7,Democratic People's Republic of Korea,NaN,NaN,9,20 to 24,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,44788,Torbay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,44789,"Bristol, City of",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,44790,Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,44791,Cornwall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


load deaths by causes

In [146]:
paths = [x for x in os.walk(os.path.join('data', 'causes'))]

file_paths = []

for folder in paths:
    for file_name in folder[2]:
        if '.csv' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data\\causes\\IHME-GBD_2019_DATA-dbded05a-1\\IHME-GBD_2019_DATA-dbded05a-1.csv',
 'data\\causes\\IHME-GBD_2019_DATA-dbded05a-2\\IHME-GBD_2019_DATA-dbded05a-2.csv',
 'data\\causes\\IHME-GBD_2019_DATA-dbded05a-3\\IHME-GBD_2019_DATA-dbded05a-3.csv']

In [147]:
causes = pd.concat([pd.read_csv(x) for x in file_paths], axis=0)

In [149]:
causes

,measure,location,sex,age,cause,metric,year,val,upper,lower
0,1,14,1,5,1022,1,2019,0.348931,0.613270,0.194354
1,1,14,2,5,1022,1,2019,0.105064,0.173193,0.061466
2,1,14,1,6,1022,1,2019,0.163581,0.248098,0.101885
3,1,14,2,6,1022,1,2019,0.061327,0.084659,0.042568
4,1,14,1,7,1022,1,2019,0.118554,0.172810,0.076119
...,...,...,...,...,...,...,...,...,...,...
24483,1,157,2,20,603,1,2019,0.042888,0.121735,0.009525
24484,1,157,2,30,603,1,2019,0.025969,0.072961,0.006559
24485,1,157,2,31,603,1,2019,0.025503,0.066125,0.006144
24486,1,157,2,32,603,1,2019,0.009223,0.034156,0.001656


load risk factors

In [12]:
paths = [x for x in os.walk(os.path.join('data', 'risk_factors'))]

file_paths = []

for folder in paths:
    for file_name in folder[2]:
        if '.csv' in file_name:
            file_paths.append(os.path.join(folder[0], file_name))

file_paths

['data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-1\\IHME-GBD_2019_DATA-cf4de7c1-1.csv',
 'data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-2\\IHME-GBD_2019_DATA-cf4de7c1-2.csv',
 'data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-3\\IHME-GBD_2019_DATA-cf4de7c1-3.csv',
 'data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-4\\IHME-GBD_2019_DATA-cf4de7c1-4.csv',
 'data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-5\\IHME-GBD_2019_DATA-cf4de7c1-5.csv',
 'data\\risk_factors\\IHME-GBD_2019_DATA-cf4de7c1-6\\IHME-GBD_2019_DATA-cf4de7c1-6.csv']

In [13]:
risks = pd.concat([pd.read_csv(x) for x in file_paths], axis=0)

In [17]:
risks[['cause', 'rei', 'location']].nunique()

cause        80
rei          50
location    204
dtype: int64

In [18]:
{v: k for k, v in location_name_to_id.items()}[location_id]

'Niue'

In [23]:
{v: k for k, v in risks_name_to_id.items()}[rei_id]

'Diet low in whole grains'

In [26]:
np.unique((
    risks
    .query('rei == @rei_id and location == @location_id')
    .sort_values(by='age')
)[['age', 'sex', 'cause', 'rei']].values, axis=1).shape

(150, 4)

load code book

In [206]:
code_book = pd.read_csv(
    os.path.join(
        'data', 'IHME_GBD_2019_CODEBOOK_Y2022M06D29.csv'
    )
).iloc[1:, 1:]


measure_name_to_id = { # create mappings
    key: int(value) 
    for key, value in code_book[['measure_name', 'measure_id']].dropna().values
}

metric_name_to_id = {
    key: int(value) 
    for key, value in code_book[['metric_name', 'metric_id']].dropna().values
}

sex_name_to_id = {
    key: int(value) 
    for key, value in code_book[['sex_label', 'sex_id']].dropna().values[1:]
}

age_name_to_id = {
    key: int(value) 
    for key, value in code_book[['age_group_name', 'age_group_id']].dropna().values
}

cause_name_to_id = {
    key: int(value) 
    for key, value in code_book[['cause_name', 'cause_id']].dropna().values
}

risks_name_to_id = {
    key: int(value) 
    for key, value in code_book[['rei_name', 'rei_id']].dropna().values
}

location_name_to_id = {
    key: int(value) 
    for key, value in code_book[['location_name', 'location_id']].dropna().values
}

name_to_id_mappings = {
    'measure_id' :measure_name_to_id,
    'metric_id' :metric_name_to_id,
    'sex_id' :sex_name_to_id,
    'age_id' :age_name_to_id,
    'cause_id' :cause_name_to_id,
    'risks_id' :risks_name_to_id,
    'location_id' :location_name_to_id,
}

id_to_name_mappings = {
    key.replace('_id', '_name'): {v: k for k, v in name_to_id_mappings[key].items()}
    for key in name_to_id_mappings.keys()
}


age_group_id_to_start_age_int = {
    '28': 0,
    '5': 1,
    '6': 5,
    '7': 10,
    '8': 15,
    '9': 20,
    '10': 25,
    '11': 30,
    '12': 35,
    '13': 40,
    '14': 45,
    '15': 50,
    '16': 55,
    '17': 60,
    '18': 65,
    '19': 70,
    '20': 75,
    '30': 80,
    '31': 85,
    '32': 90,
    '235': 95,
    '33': 95,
    '44': 100,
    '45': 105,
    '148': 110
}

age_group_name_to_start_age_int = {
    '<1 year': 0,
    '1 to 4': 1,
    '5 to 9': 5,
    '10 to 14': 10,
    '15 to 19': 15,
    '20 to 24': 20,
    '25 to 29': 25,
    '30 to 34': 30,
    '35 to 39': 35,
    '40 to 44': 40,
    '45 to 49': 45,
    '50 to 54': 50,
    '55 to 59': 55,
    '60 to 64': 60,
    '65 to 69': 65,
    '70 to 74': 70,
    '75 to 79': 75,    
    '80 to 84': 80,
    '85 to 89': 85,
    '90 to 94': 90,
    '95 plus': 95,
    '95 to 99': 95,
    '100 to 104': 100,
    '105 to 109': 105,
    '110 plus': 110,    
}


## Preprocessing

location harmonization

In [134]:
risks.dtypes

measure       int64
location      int64
sex           int64
age           int64
cause         int64
rei           int64
metric        int64
year          int64
val         float64
upper       float64
lower       float64
dtype: object

In [135]:
risks.location.nunique(), causes.location.nunique()

(204, 204)

In [136]:
causes.dtypes

measure       int64
location      int64
sex           int64
age           int64
cause         int64
metric        int64
year          int64
val         float64
upper       float64
lower       float64
dtype: object

In [137]:
len(set(risks.location.values).intersection(set(causes.location.values)))

204

In [75]:
population = population[population.columns[~population.columns.str.contains('_name')]]

In [76]:
population.dtypes

location_id       int64
sex_id            int64
age_group_id      int64
year_id           int64
measure_id        int64
metric_id         int64
val             float64
upper           float64
lower           float64
dtype: object

In [154]:
life_expectancy = life_expectancy[life_expectancy.columns[~life_expectancy.columns.str.contains('_name')]]

In [155]:
life_expectancy.dtypes

measure       int64
location      int64
sex           int64
age           int64
metric        int64
year          int64
val         float64
upper       float64
lower       float64
dtype: object

In [79]:
population.location_id.nunique(), life_expectancy.location_id.nunique()

(675, 675)

In [68]:
len(set(risks.location.values).intersection(set(population.location_id.values)))

204

In [ ]:
len(set(risks.location.values).intersection(set(life_expectancy.location_id.values)))

In [158]:
life_expectancy.location.nunique()

204

In [83]:
population_source = population.copy()
life_expectancy_source = life_expectancy.copy()

In [84]:
population = population.query('location_id in @risks.location.unique()')
life_expectancy = life_expectancy.query('location_id in @risks.location.unique()')

In [85]:
population.location_id.nunique(), life_expectancy.location_id.nunique()

(204, 204)

age harmonization

In [86]:
(
    causes.age.nunique(),
    risks.age.nunique(),
    population.age_group_id.nunique(),
    life_expectancy.age_group_id.nunique(),
)

(21, 21, 47, 24)

In [178]:
len(set(causes.age.unique()).intersection(
    set(risks.age.unique())
))

21

In [87]:
len(set(causes.age.unique()).intersection(
    set(population.age_group_id.unique())
))

21

In [88]:
len(set(causes.age.unique()).intersection(
    set(life_expectancy.age_group_id.unique())
))

20

In [89]:
len(set(life_expectancy.age_group_id.unique()).intersection(
    set(population.age_group_id.unique())
))

20

In [90]:
np.array([id_to_name_mappings['age_name'][x] for x in set(causes.age.unique()).difference(
    set(life_expectancy.age_group_id.unique())
)])

array(['95 plus'], dtype='<U7')

In [91]:
np.array([id_to_name_mappings['age_name'][x] for x in risks.age.unique()])

array(['95 plus', '25 to 29', '30 to 34', '35 to 39', '40 to 44',
       '45 to 49', '50 to 54', '55 to 59', '60 to 64', '65 to 69',
       '70 to 74', '75 to 79', '80 to 84', '85 to 89', '90 to 94',
       '20 to 24', '10 to 14', '15 to 19', '<1 year', '1 to 4', '5 to 9'],
      dtype='<U8')

In [92]:
np.array([life_expectancy_age_group_id_to_name[x] for x in life_expectancy.age_group_id.unique()])

array(['25 to 29', '30 to 34', '35 to 39', '40 to 44', '110 plus',
       '45 to 49', '50 to 54', '55 to 59', '60 to 64', '65 to 69',
       '70 to 74', '75 to 79', '<1 year', '80 to 84', '85 to 89',
       '90 to 94', '95 to 99', '100 to 104', '105 to 109', '1 to 4',
       '5 to 9', '10 to 14', '15 to 19', '20 to 24'], dtype='<U10')

In [93]:
population = population.query('age_group_id in @risks.age.unique()')

In [ ]:
risks.age.nunique(), causes.age.nunique(), population.age_group_id.nunique(), life_expectancy.age_group_id.nunique()

In [95]:
## we're will using life_expectancy for check accuracy our estimation thus leave age groups in this table as is 

sex harmonization

In [96]:
population.sex_id.unique()

array([1, 2, 3], dtype=int64)

In [97]:
life_expectancy.sex_id.unique()

array([1, 2], dtype=int64)

In [73]:
causes.sex.unique()

array([1, 2], dtype=int64)

In [74]:
risks.sex.unique()

array([1, 2], dtype=int64)

In [100]:
population = population.query('sex_id in @risks.sex.unique()')
population.sex_id.unique()

array([1, 2], dtype=int64)

convert age group id to age group start age

In [162]:
age_id_to_name = {v: k for k, v in age_name_to_id.items()}

In [103]:
population['age_group_start'] = (
    population['age_group_id']
    .copy()
    .map(age_id_to_name)
    .map(age_group_name_to_start_age_int)
)
population['age_group_start'] = population['age_group_start'].astype('int')

In [214]:
life_expectancy['age_group_start'] = (
    life_expectancy['age_id'].astype('str')
    .copy()
    .map(age_group_id_to_start_age_int)
)

In [215]:
life_expectancy.isna().sum() / len(life_expectancy)

measure_id         0.0
measure_name       0.0
location_id        0.0
location_name      0.0
sex_id             0.0
sex_name           0.0
age_id             0.0
age_name           0.0
metric_id          0.0
metric_name        0.0
year               0.0
val                0.0
upper              0.0
lower              0.0
age_group_start    0.0
dtype: float64

In [154]:
causes['age_group_start'] = (
    causes['age']
    .copy()
    .map(age_id_to_name)
    .map(age_group_name_to_start_age_int)
)

In [139]:
causes.isna().sum() / len(causes)

measure            0.0
location           0.0
sex                0.0
age                0.0
cause              0.0
metric             0.0
year               0.0
val                0.0
upper              0.0
lower              0.0
age_group_start    0.0
dtype: float64

In [155]:
risks['age_group_start'] = (
    risks['age']
    .copy()
    .map(age_id_to_name)
    .map(age_group_name_to_start_age_int)
)

In [141]:
risks.isna().sum() / len(risks)

measure            0.0
location           0.0
sex                0.0
age                0.0
cause              0.0
rei                0.0
metric             0.0
year               0.0
val                0.0
upper              0.0
lower              0.0
age_group_start    0.0
dtype: float64

In [156]:
risks.drop('age', axis=1, inplace=True)
causes.drop('age', axis=1, inplace=True)

In [110]:
population.drop('age_group_id', axis=1, inplace=True)

In [216]:
life_expectancy.drop('age_id', axis=1, inplace=True)

Columns names harmonization

In [157]:
risks.columns = [
    f'{x}_id' 
    if x in ['measure', 'location', 'sex', 'cause', 'rei', 'metric'] 
    else x 
    for x in risks.columns
]

causes.columns = [
    f'{x}_id' 
    if x in ['measure', 'location', 'sex', 'cause', 'rei', 'metric'] 
    else x  
    for x in causes.columns
]

In [219]:
life_expectancy = life_expectancy[life_expectancy.columns[~life_expectancy.columns.str.contains('_name')]]

In [220]:
life_expectancy.columns

Index(['measure_id', 'location_id', 'sex_id', 'metric_id', 'year', 'val',
       'upper', 'lower', 'age_group_start'],
      dtype='object')

Filtering only most detailed causes and risk factors

In [158]:
aggregate_causes = np.array(
    [
        x
        for x in causes.cause_id.unique()
        if 'Total' in id_to_name_mappings['cause_name'][x]
    ]
)

In [159]:
[id_to_name_mappings['cause_name'][x] for x in aggregate_causes]

[]

In [110]:
causes = causes.query('cause_id not in @aggregate_causes')
risks = risks.query('cause_id not in @aggregate_causes')

Saving preprocessed data

In [160]:
risks.to_csv(os.path.join('data', 'preprocessed', 'risks.csv'), index=False)
causes.to_csv(os.path.join('data', 'preprocessed', 'causes.csv'), index=False)

In [111]:
population.to_csv(os.path.join('data', 'preprocessed', 'population.csv'), index=False)


In [221]:
life_expectancy.to_csv(os.path.join('data', 'preprocessed', 'life_expectancy.csv'), index=False)

In [113]:
np.sort(life_expectancy.age_group_start.unique())

array([  0,   1,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,
        60,  65,  70,  75,  80,  85,  90,  95, 100, 105, 110], dtype=int64)

In [114]:
np.sort(population.age_group_start.unique())

array([ 0,  1,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75,
       80, 85, 90, 95])

In [161]:
np.sort(causes.age_group_start.unique())

array([ 0,  1,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75,
       80, 85, 90, 95], dtype=int64)

In [162]:
np.sort(risks.age_group_start.unique())

array([ 0,  1,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75,
       80, 85, 90, 95], dtype=int64)

## Estimation life expectancy

Functions definitions

In [32]:
[x for x in risks_name_to_id.keys() if 'All' in x]

['All risk factors']

In [34]:
[x for x in cause_name_to_id.keys() if 'All' in x]

['All causes']

In [35]:
risks_name_to_id['All risk factors'], cause_name_to_id['All causes']

(169, 294)

In [70]:
def risk_factors_impact_to_mortality_estimator(
    risks_located: pd.DataFrame,
    all_causes_id: int=294,
    all_risks_id: int=169,
) -> pd.DataFrame:

    """
    Function to estimate impact of each risk factor in deaths attributed by all causes
    as their share in sum of most detailed mortality multiplied by 
    sum of deaths attributed at least one risk factor
    """
    # create groupped dataframe with summarized risk impact by
    # sex age and risk factors
    all_causes_risk_detailed = (
        risks_located
        .query(
            'cause_id == @all_causes_id'
            ' and rei_id != @all_risks_id'
        )
        .groupby(by=['sex_id', 'age_group_start', 'rei_id'])
        [['val']].sum()
    )

    all_causes_risk_detailed_sum = all_causes_risk_detailed.val.sum()

    all_causes_risk_detailed_share = all_causes_risk_detailed.copy()
    
    all_causes_risk_detailed_share['val'] = (
        all_causes_risk_detailed['val']
        /
        all_causes_risk_detailed_sum
    )

    at_least_one_risk_factor_deaths_sum = (
        risks_located.query(
            'rei_id == @all_risks_id'
            ' and cause_id == @all_causes_id'
        )
        ['val']
        .sum()
    )

    risk_factors_impact = all_causes_risk_detailed.copy()

    risk_factors_impact['val'] = (
        all_causes_risk_detailed_share['val'].values
        *
        at_least_one_risk_factor_deaths_sum
    )

    return risk_factors_impact


def data_interpolate_inside_age_groups(
    data: pd.DataFrame,
    start_age_column: str,
    columns_to_interpolate: list,
    val_columns: list,
    columns_for_subsample: list,
    columns_for_subsample_is_string: bool=False,
    age_max: int=110,
    age_max_value: int=0,
    method='linear',
    divide=True,
    dividers = {
        0: 1,
        1: 4,
        5: 5,
        10: 5,
        15: 5,
        20: 5,
        25: 5,
        30: 5,
        35: 5,
        40: 5,
        45: 5,
        50: 5,
        55: 5,
        60: 5,
        65: 5,
        70: 5,
        75: 5,
        80: 5,
        85: 5,
        90: 5,
        95: 15,
    },
    next_similar_values = {
    1: 4,
    5: 9,
    10: 14,
    15: 19,
    20: 24,
    25: 29,
    30: 34,
    35: 39,
    40: 44,
    45: 49,
    50: 54,
    55: 59,
    60: 64,
    65: 69,
    70: 74,
    75: 79,
    80: 84,
    85: 89,
    90: 94,
    95: 109,
},
) -> pd.DataFrame:

    """
    Linear interpolate data inside age groups 
    return data with values converted from many years 
    bins to single year bins
    """ 
    variants_filters = pd.DataFrame(
        index=pd.MultiIndex.from_product(
            [
                data[column].unique()
                for column in columns_for_subsample
            ],
            names=columns_for_subsample
        )
    ).reset_index()

    result = []

    for variant in variants_filters.iterrows():
        if columns_for_subsample_is_string == True:
            query_text = ' and '.join(
                [f'{column} == "{variant[1][column]}"' for column in columns_for_subsample]
            )
        else:
            query_text = ' and '.join(
                [f'{column} == {variant[1][column]}' for column in columns_for_subsample]
            )
        subsample_for_interpolation = (
            data.query(query_text)
            [[start_age_column] + columns_to_interpolate]
            .set_index(start_age_column)
            .sort_index()
        )
        
        subsample_for_interpolation = (
            subsample_for_interpolation
            .join(
                pd.DataFrame(
                    index=pd.Index(range(0, age_max + 1, 1))
                ),
                how='right'
            )
            .join(
                pd.DataFrame(
                    {'dividers': dividers.values()},
                    index=dividers.keys()
                )
            )
        )
        
        if divide == True:
            divided_subsample_for_interpolation_val = pd.concat(
                [
                    subsample_for_interpolation[column]
                    /
                    subsample_for_interpolation['dividers']
                    for column in val_columns
                ],
                axis=1
            )
            divided_subsample_for_interpolation_val.columns = val_columns

            divided_subsample_for_interpolation = divided_subsample_for_interpolation_val.copy()
            
            subsample_for_interpolation[columns_to_interpolate] = (
                divided_subsample_for_interpolation[columns_to_interpolate]
            )

            (
                subsample_for_interpolation
                .loc[
                    next_similar_values.values(),
                    columns_to_interpolate
                ]
            ) = (
                subsample_for_interpolation
                .loc[
                    next_similar_values.keys(),
                    columns_to_interpolate
                ]
                .values
            )

        subsample_for_interpolation[-1:] = subsample_for_interpolation[-1:].fillna(age_max_value).values
        
        interpolated_data_subsample = pd.concat(
            [
                subsample_for_interpolation[column].interpolate(method=method)
                for column in columns_to_interpolate
            ],
            axis=1
        )

        interpolated_data_subsample[variant[1].index] = variant[1].values

        result.append(interpolated_data_subsample)

    result = pd.concat(result, axis=0)

    return result


def life_expectancy_from_population_and_deats(
    data: pd.DataFrame,
    age_group_start_column: str,
    deaths_column: str,
    population_column: str,
    radix: int=100_000,
    last_age_group_years: int=5,
) -> pd.DataFrame:
    """
    calculating life expectancy by population and deaths
    based on guideline  
    https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/methodologies/guidetocalculatingnationallifetables
    """

    life_table = data.copy()

    # Calculate q_x - likelihood or risk of death having reached age x
    divider = (
        (0.5 * life_table[deaths_column].values)
        +
        life_table[population_column].values
    )

    q_x_with_ui = (
        life_table[deaths_column].values
        /
        divider
    )

    life_table['q_x_val'] = q_x_with_ui
    
    # Calculate l_x - number of people surviving to age x
    life_table.sort_values(by=age_group_start_column, inplace=True)

    life_table['l_x_val'] = np.nan

    life_table.loc[life_table.index[0], 'l_x_val'] = radix

    for i in range(1, len(life_table.index), 1):
        substrahend = (
            life_table.loc[life_table.index[i - 1], 'l_x_val']
            *
            life_table.loc[life_table.index[i - 1], 'q_x_val']
        )

        current_age_lx = (
            life_table.loc[life_table.index[i - 1], 'l_x_val']
            -
            substrahend
        )

        life_table.loc[life_table.index[i], 'l_x_val'] = current_age_lx
    
    # Calculate L_x - the number of person years lived at each individual age group
    life_table['L_x_val'] = np.nan

    for i in range(0, len(life_table.index) - 1, 1):

        curr_age_group_years = (
            life_table.loc[life_table.index[i + 1], age_group_start_column]
            -
            life_table.loc[life_table.index[i], age_group_start_column]
        )

        current_age_L_x = (
            life_table.loc[life_table.index[i], 'l_x_val']
            +
            life_table.loc[life_table.index[i + 1], 'l_x_val']
        )

        life_table.loc[life_table.index[i], 'L_x_val'] = curr_age_group_years * (current_age_L_x / 2)

    life_table.loc[life_table.index[-1], 'L_x_val'] = (
        last_age_group_years * (life_table.loc[life_table.index[-1], 'l_x_val'] / 2)
    )
    
    # Calculate T_x - total number of years lived
    life_table['T_x_val'] = np.nan

    for i in range(0, len(life_table.index), 1):

        curr_age_group_sum = life_table.loc[life_table.index[i]:, 'L_x_val'].sum()

        life_table.loc[life_table.index[i], 'T_x_val'] = curr_age_group_sum
            
    # Calculate E_x - period expectation of life at exact age x
    E_x = (
        life_table['T_x_val'].values
        /
        life_table['l_x_val'].values
    )

    life_table['E_x_val'] = E_x

    return life_table


def life_expectancy_risk_removing_from_deaths(
    causes_located: pd.DataFrame,
    risks_located: pd.DataFrame,
    population_located: pd.DataFrame,
    all_risk_factors_id: int=169,
    all_causes_id: int=294,
    risks_to_remove: list=None,    
    age_column: str='age_group_start',
    sex_column: str='sex_id',
    le_estimation: str='val',
    population_estimation: str='val',
    mortality_estimation: str='val',
    risk_impact_deaths_prefix: str='rei_impact_deaths_',
    mortality_suffix: str='_deaths',
    population_suffix: str='_population',
    interpolation_method='linear',
    interpolation_age_max: int=111,
    age_result_trunc: int=95,
    interpolation_age_max_value: int=1,
    radix: int=100_000,
    le_calculation_last_age_group_years: int=5,
    disable_verbocity=True,
) -> pd.DataFrame:

    mortality = (
        causes_located.query('cause_id == @all_causes_id')
        .set_index([age_column, sex_column]).sort_index()
        [[mortality_estimation]]
    )

    mortality_columns_names = [mortality_estimation + mortality_suffix]
    population_columns_names = [population_estimation + population_suffix]

    population_located.columns = [
        f'{x}{population_suffix}'
        if x == population_estimation
        else x
        for x in population_located.columns
    ]

    mortality.columns = [
        f'{x}{mortality_suffix}'
        if x == mortality_estimation 
        else x
        for x in mortality.columns
    ]

    life_table = (
        population_located
        .copy()
        .set_index([age_column, sex_column])
        [population_columns_names]
        .join(
            mortality.copy()
            [mortality_columns_names],            
        )
        [mortality_columns_names + population_columns_names]
        .sort_index()
        .reset_index()
    )

    if risks_to_remove is not None:        
        risk_factors_impact_to_deaths_by_causes = risk_factors_impact_to_mortality_estimator(
            risks_located=risks_located,
        )

        risks_impact = (
            risk_factors_impact_to_deaths_by_causes
            .query('rei_id in @risks_to_remove')
            .groupby(by=[age_column, sex_column])
            [['val']]
            .sum()
            .join(
                pd.DataFrame(
                    index=pd.MultiIndex.from_product(
                        [risks_located[age_column].unique(), risks_located[sex_column].unique()],
                        names=[age_column, sex_column],
                    )
                )
                .sort_index(),
                how='outer'
            ).fillna(0)
        )

        risks_impact.columns = [
            f'{risk_impact_deaths_prefix}_{x}' for x in risks_impact.columns
        ]

        life_table_risk_excluded = life_table.copy()

        life_table_risk_excluded = (
            life_table_risk_excluded
            .copy()
            .set_index([age_column, sex_column])
            .join(
                risks_impact,
                how='left'
            )
        )
        
        life_table_risk_excluded[mortality_estimation + mortality_suffix] = (
            life_table_risk_excluded[mortality_estimation + mortality_suffix]
            -
            life_table_risk_excluded[f'{risk_impact_deaths_prefix}_val']
        )
        
        life_table_risk_excluded = (
            life_table_risk_excluded
            [mortality_columns_names + population_columns_names]
            .reset_index()
        )

    if risks_to_remove is None:
        life_table_risk_excluded = life_table.copy()

    life_table_interpolated = data_interpolate_inside_age_groups(
        data=life_table_risk_excluded,
        start_age_column=age_column,
        columns_to_interpolate=mortality_columns_names + population_columns_names,
        val_columns=mortality_columns_names + population_columns_names,
        columns_for_subsample=[sex_column],
        columns_for_subsample_is_string=False,
        age_max=interpolation_age_max,
        age_max_value=interpolation_age_max_value,
        divide=True,
        method=interpolation_method,
    )

    life_table_interpolated.reset_index(inplace=True)

    life_table_interpolated.columns = (
        [age_column]
        +
        mortality_columns_names
        +
        population_columns_names
        +
        [sex_column]
    )

    life_table_interpolated = life_table_interpolated[
        [age_column]
        +
        [sex_column]
        +
        mortality_columns_names
        +
        population_columns_names
    ].query(f'{age_column} <= {interpolation_age_max}')

    result = pd.concat(
        [
            life_expectancy_from_population_and_deats(
                data=life_table_interpolated.query('sex_id == @sex_id'),
                age_group_start_column='age_group_start',
                deaths_column=f'{mortality_estimation}{mortality_suffix}',
                population_column=f'{population_estimation}{population_suffix}',
                radix=radix,
                last_age_group_years=le_calculation_last_age_group_years,
            )
            for sex_id in life_table_interpolated[sex_column].unique()
        ]
    )
    result = result.iloc[:, np.where(result.columns.duplicated(keep='first') == False)[0]]

    result.columns = (
        result.columns.str.replace(
            'E_x_val',
            f'E_x_{le_estimation}'

        )
    )

    result = (
        result
        .copy()
        [
            [age_column, sex_column]
            +
            [f'E_x_{le_estimation}']
        ]
    ).reset_index(drop=True)

    return result.query(f'{age_column} <= @age_result_trunc')


Loading code book

In [5]:
code_book = pd.read_csv(
    os.path.join(
        'data', 'IHME_GBD_2019_CODEBOOK_Y2022M06D29.csv'
    )
).iloc[1:, 1:]


measure_name_to_id = { # create mappings
    key: int(value) 
    for key, value in code_book[['measure_name', 'measure_id']].dropna().values
}

metric_name_to_id = {
    key: int(value) 
    for key, value in code_book[['metric_name', 'metric_id']].dropna().values
}

sex_name_to_id = {
    key: int(value) 
    for key, value in code_book[['sex_label', 'sex_id']].dropna().values[1:]
}

age_name_to_id = {
    key: int(value) 
    for key, value in code_book[['age_group_name', 'age_group_id']].dropna().values
}

cause_name_to_id = {
    key: int(value) 
    for key, value in code_book[['cause_name', 'cause_id']].dropna().values
}

risks_name_to_id = {
    key: int(value) 
    for key, value in code_book[['rei_name', 'rei_id']].dropna().values
}

location_name_to_id = {
    key: int(value) 
    for key, value in code_book[['location_name', 'location_id']].dropna().values
}

name_to_id_mappings = {
    'measure_id' :measure_name_to_id,
    'metric_id' :metric_name_to_id,
    'sex_id' :sex_name_to_id,
    'age_id' :age_name_to_id,
    'cause_id' :cause_name_to_id,
    'risks_id' :risks_name_to_id,
    'location_id' :location_name_to_id,
}

id_to_name_mappings = {
    key.replace('_id', '_name'): {v: k for k, v in name_to_id_mappings[key].items()}
    for key in name_to_id_mappings.keys()
}


age_group_name_to_start_age_int = {
    '<1 year': 0,
    '1 to 4': 1,
    '5 to 9': 5,
    '10 to 14': 10,
    '15 to 19': 15,
    '20 to 24': 20,
    '25 to 29': 25,
    '30 to 34': 30,
    '35 to 39': 35,
    '40 to 44': 40,
    '45 to 49': 45,
    '50 to 54': 50,
    '55 to 59': 55,
    '60 to 64': 60,
    '65 to 69': 65,
    '70 to 74': 70,
    '75 to 79': 75,    
    '80 to 84': 80,
    '85 to 89': 85,
    '90 to 94': 90,
    '95 plus': 95,
    '95 to 99': 95,
    '100 to 104': 100,
    '105 to 109': 105,
    '110 plus': 110,    
}

age_group_id_to_start_age_int = {
    '28': 0,
    '5': 1,
    '6': 5,
    '7': 10,
    '8': 15,
    '9': 20,
    '10': 25,
    '11': 30,
    '12': 35,
    '13': 40,
    '14': 45,
    '15': 50,
    '16': 55,
    '17': 60,
    '18': 65,
    '19': 70,
    '20': 75,
    '30': 80,
    '31': 85,
    '32': 90,
    '235': 95
}

Loading lists of risk factors

In [6]:
risks_names_manageable = pd.read_csv(
    os.path.join(
        'data', 
        'preprocessed',
        'risks_names_manageable.csv'
    )
)

risks_names_manageable = [x[0] for x in risks_names_manageable.values]

Loading preporcessed data

In [28]:
risks = pd.read_csv(os.path.join('data', 'preprocessed', 'risks.csv'))
causes = pd.read_csv(os.path.join('data', 'preprocessed', 'causes.csv'))
population = pd.read_csv(os.path.join('data', 'preprocessed', 'population.csv'))
life_expectancy = pd.read_csv(os.path.join('data', 'preprocessed', 'life_expectancy.csv'))

In [29]:
rei_id, location_id = (113, 374)

In [30]:
(
    risks
    .query('rei_id == @rei_id and location_id == @location_id')
    .sort_values(by='age_group_start')
)

,measure_id,location_id,sex_id,cause_id,rei_id,metric_id,year,val,upper,lower,age_group_start
2541757,1,374,1,494,113,1,2019,0.000044,0.000107,0.000008,25
1715583,1,374,2,493,113,1,2019,0.000616,0.001173,0.000162,25
2200499,1,374,2,294,113,1,2019,0.000850,0.001517,0.000327,25
2200457,1,374,1,294,113,1,2019,0.002519,0.004748,0.000897,25
2541776,1,374,2,494,113,1,2019,0.000080,0.000182,0.000013,25
...,...,...,...,...,...,...,...,...,...,...,...
2515144,1,374,1,294,113,1,2019,0.005279,0.007302,0.002320,95
2226229,1,374,2,494,113,1,2019,0.000539,0.000986,0.000128,95
2226209,1,374,1,494,113,1,2019,0.000189,0.000341,0.000041,95
1184083,1,374,1,587,113,1,2019,0.000427,0.000793,0.000119,95


In [41]:
risk_factors_impact = risk_factors_impact_to_mortality_estimator(
    risks.query('location_id == @location_id')
).reset_index().age_group_start.nunique()

In [54]:
risk_factors_impact.columns

Index(['sex_id', 'age_group_start', 'rei_id', 'val'], dtype='object')

In [55]:
risks_to_remove = [risks_name_to_id['Smoking']]
risks_to_remove

[99]

In [61]:
risks.sex_id.unique()

array([1, 2], dtype=int64)

In [ ]:
pd.DataFrame(
    index=pd.MultiIndex.from_product(
    [risks.age_group_start.unique(), risks.sex_id.unique()],
    names=['age_group_start', 'sex_id'],
)
).sort_index()

In [49]:
risk_factors_impact.age_group_start.nunique()

21

### Estimation for one choosen country

In [7]:
country = np.random.choice(population.location_id.unique())
print(f'Country: {id_to_name_mappings["location_name"][country]}')
print(f'Populatiom: {round(population.query("location_id == @country").val.sum() / 10**6, 2)} M')

Country: United Arab Emirates
Populatiom: 9.24 M


In [8]:
[x for x in location_name_to_id.keys() if 'Russ' in x]

['Russian Federation']

In [9]:
country = location_name_to_id['Russian Federation']
print(f'Populatiom: {round(population.query("location_id == @country").val.sum() / 10**6, 2)} M')

Populatiom: 146.72 M


In [10]:
risks_located = risks.query('location_id == @country')
causes_located = causes.query('location_id == @country')
population_located = population.query('location_id == @country')
life_expectancy_located = life_expectancy.query('location_id == @country')

#### Example with calculation risk factors impact

In [11]:
all_causes_id = cause_name_to_id['All causes']
all_risks_id = risks_name_to_id['All risk factors']
all_causes_id, all_risks_id

(294, 169)

In [14]:
# sum all deaths in year
causes_located.query('cause_id == @all_causes_id').val.sum()

1788286.2644205429

In [15]:
# sum of deaths detailed attributed by causes
causes_located.query('cause_id != @all_causes_id').val.sum()


1788286.2644205436

In [16]:
# how many all caused deaths have attribution at least one risk factor 
risks_located.query(
    'rei_id == @all_risks_id'
    ' and cause_id == @all_causes_id'
).val.sum()

1243375.8309223247

In [17]:
# sum of deaths detailed attributied by risk facotrs
# and have attribution at least one risk factor
risks_located.query(
    'rei_id == @all_risks_id'
    ' and cause_id != @all_causes_id'
).val.sum()

1243375.8309223242

In [18]:
# sum of deaths detailed attributed 
# to each risk factor by each cause
# value more than two time larger because one death might be attributed more than one risk factor
risks_located.query(
    'rei_id != @all_risks_id'
    ' and cause_id != @all_causes_id'
).val.sum()

2841443.9044831092

In [ ]:
# Lets estimate impact of each risk factor in deaths attributed by all causes
# as their share in sum of most detailed mortality multiplied by 
# sum of deaths attributed at least one risk factor

In [24]:
# create groupped dataframe with summarized risk impact by
all_causes_risk_detailed = (
    (
        risks_located
        .query(
            'cause_id == @all_causes_id'
            ' and rei_id != @all_risks_id'
        )
        .groupby(by=['sex_id', 'age_group_start', 'rei_id'])
    )
    [['val']]
    .sum()
    .reset_index()
)

In [25]:
# check that summary all risks impact equal 
# sum of deaths most detailed attributed 
# to each risk factor and each cause
all_causes_risk_detailed.val.sum()

2841443.904483109

In [28]:
# calculate shares of impact in most detailed attribution
all_causes_risk_detailed_share = (
    all_causes_risk_detailed[['val']]
    /
    all_causes_risk_detailed.val.sum()
)


all_causes_risk_detailed_share = pd.concat(
    [
        all_causes_risk_detailed[['sex_id', 'age_group_start', 'rei_id']],
        all_causes_risk_detailed_share
    ],
    axis=1
)

In [29]:
# how many all caused deaths have attribution at least one risk factor 
at_least_one_risk_factor_deaths_sum = risks_located.query(
    'rei_id == @all_risks_id'
    ' and cause_id == @all_causes_id'
).val.sum()

at_least_one_risk_factor_deaths_sum

1243375.8309223247

In [30]:
# now we are just multiply obtained most detailed sum's shares
# by the sum of deaths attributed at least one risk_factor
# and thus gained our estimation of the each risk factor impact to the mortality
risk_factors_impact = (
    all_causes_risk_detailed_share[['val']]
    *
    at_least_one_risk_factor_deaths_sum
)

risk_factors_impact = pd.concat(
    [
        all_causes_risk_detailed[['sex_id', 'age_group_start', 'rei_id']],
        risk_factors_impact
    ],
    axis=1
)

In [31]:
# Check that sum equal sum of deaths attributed at least one risk factor
risk_factors_impact.val.sum()

1243375.8309223247

In [33]:
risks_located.query(
    'rei_id == @all_risks_id'
    ' and cause_id != @all_causes_id'
).val.sum()

1243375.8309223242

In [34]:
risk_factors_impact

,sex_id,age_group_start,rei_id,val
0,1,0,83,7.937257
1,1,0,84,6.244888
2,1,0,88,0.000000
3,1,0,90,0.000000
4,1,0,91,0.000000
...,...,...,...,...
1561,2,95,337,3.723342
1562,2,95,338,942.631284
1563,2,95,341,910.115468
1564,2,95,367,3391.127781


#### Example with calculation life expectancy from deaths and population

In [ ]:
# for estimation life expectancy we will using life tables approach

In [13]:
life_expectancy_located_interpolated = data_interpolate_inside_age_groups(
    life_expectancy_located[['age_group_start', 'sex_id', 'val', 'upper', 'lower']],
    columns_to_interpolate=['val', 'upper', 'lower'],
    columns_for_subsample=['sex_id'],
    columns_for_subsample_is_string=False,
    val_columns=['val'],
    start_age_column='age_group_start',
    age_max=111,
    divide=False,
    age_max_value=1,
    next_similar_values = {
        1: 4,
        5: 9,
        10: 14,
        15: 19,
        20: 24,
        25: 29,
        30: 34,
        35: 39,
        40: 44,
        45: 49,
        50: 54,
        55: 59,
        60: 64,
        65: 69,
        70: 74,
        75: 79,
        80: 84,
        85: 89,
        90: 94,
        95: 99,
        100: 104,
        105: 109,
        110: 111,
    }
)

life_expectancy_located_interpolated.reset_index(inplace=True)

life_expectancy_located_interpolated.columns = (
    ['age_group_start']
    +
    list(life_expectancy_located_interpolated.columns[1:])
)

In [14]:
life_expectancy_located_interpolated

,age_group_start,val,upper,lower,sex_id
0,0,67.955377,70.436108,65.558052,1
1,1,67.347908,69.833444,64.956161,1
2,2,66.371871,68.858778,63.980663,1
3,3,65.395833,67.884112,63.005166,1
4,4,64.419796,66.909447,62.029668,1
...,...,...,...,...,...
219,107,1.511234,1.583642,1.456360,2
220,108,1.450179,1.514746,1.400827,2
221,109,1.389125,1.445850,1.345295,2
222,110,1.328070,1.376954,1.289763,2


In [17]:
le_calculated = life_expectancy_risk_removing_from_deaths(
    causes_located=causes_located,
    risks_located=risks_located,
    population_located=population_located,
    all_risk_factors_id=169,
    all_causes_id=294,
    risks_to_remove=None,    
    age_column='age_group_start',
    sex_column='sex_id',
    le_estimation='val',
    risk_impact_deaths_prefix='rei_impact_deaths_',
    mortality_suffix='_deaths',
    population_suffix='_population',
    interpolation_method='linear',
    interpolation_age_max=111,
    age_result_trunc=95,
    interpolation_age_max_value=1,
    radix=100_000,
    le_calculation_last_age_group_years=5,
    disable_verbocity=True,
)

In [18]:
le_calculated_risk_excluded = life_expectancy_risk_removing_from_deaths(
    causes_located=causes_located,
    risks_located=risks_located,
    population_located=population_located,
    all_risk_factors_id=169,
    all_causes_id=294,
    risks_to_remove=[risks_name_to_id[x] for x in risks_names_manageable],    
    age_column='age_group_start',
    sex_column='sex_id',
    le_estimation='val',
    risk_impact_deaths_prefix='rei_impact_deaths_',
    mortality_suffix='_deaths',
    population_suffix='_population',
    interpolation_method='linear',
    interpolation_age_max=111,
    age_result_trunc=95,
    interpolation_age_max_value=1,
    radix=100_000,
    le_calculation_last_age_group_years=5,
    disable_verbocity=True,
)

In [19]:
sex_id = 1


px.line(
    le_calculated
    .set_index(['age_group_start', 'sex_id'])
    .join(
        le_calculated_risk_excluded
        .set_index(['age_group_start', 'sex_id']),
        rsuffix='_rei_excluded'
    )
    .join(
        life_expectancy_located_interpolated
        .set_index(['age_group_start', 'sex_id']),
        rsuffix='_source'
    )
    .query('sex_id == @sex_id')
    .reset_index()
    .set_index('age_group_start')
    [['val', 'E_x_val', 'E_x_val_rei_excluded']]
)


### Calculate for all countries and all manageable risk factors

In [73]:
le_calculated_all_countries = []

le_calculated_risk_excluded_all_countries = []

for location_id in tqdm(risks.location_id.unique()):

    causes_located = causes.query('location_id == @location_id')
    risks_located = risks.query('location_id == @location_id')
    population_located = population.query('location_id == @location_id')

    le_calculated = life_expectancy_risk_removing_from_deaths(
        causes_located=causes_located,
        risks_located=risks_located,
        population_located=population_located,
        all_risk_factors_id=169,
        all_causes_id=294,
        risks_to_remove=None,    
        age_column='age_group_start',
        sex_column='sex_id',
        le_estimation='val',
        risk_impact_deaths_prefix='rei_impact_deaths_',
        mortality_suffix='_deaths',
        population_suffix='_population',
        interpolation_method='linear',
        interpolation_age_max=111,
        age_result_trunc=95,
        interpolation_age_max_value=1,
        radix=100_000,
        le_calculation_last_age_group_years=5,
        disable_verbocity=True,
    )
    le_calculated['location_id'] = location_id
    le_calculated['excluded_rei_id'] = np.nan
    le_calculated_all_countries.append(le_calculated)

    for rei_id in [risks_name_to_id[x] for x in risks_names_manageable]:

        le_calculated_risk_excluded = life_expectancy_risk_removing_from_deaths(
            causes_located=causes_located,
            risks_located=risks_located,
            population_located=population_located,
            all_risk_factors_id=169,
            all_causes_id=294,
            risks_to_remove=[rei_id],    
            age_column='age_group_start',
            sex_column='sex_id',
            le_estimation='val',
            risk_impact_deaths_prefix='rei_impact_deaths_',
            mortality_suffix='_deaths',
            population_suffix='_population',
            interpolation_method='linear',
            interpolation_age_max=111,
            age_result_trunc=95,
            interpolation_age_max_value=1,
            radix=100_000,
            le_calculation_last_age_group_years=5,
            disable_verbocity=True,
        )

        le_calculated_risk_excluded['location_id'] = location_id

        le_calculated_risk_excluded['excluded_rei_id'] = rei_id

        le_calculated_risk_excluded_all_countries.append(le_calculated_risk_excluded)

  0%|          | 0/204 [00:00<?, ?it/s]

100%|██████████| 204/204 [22:05<00:00,  6.50s/it]


In [74]:
le_calculated_all_countries_df = pd.concat(le_calculated_all_countries, axis=0)
le_calculated_risk_excluded_all_countries_df = pd.concat(le_calculated_risk_excluded_all_countries, axis=0)

In [75]:
le_mean_absolute_error = []
for location_id in tqdm(le_calculated_all_countries_df.location_id.unique()):

    le_mean_absolute_error.append(
        (
            life_expectancy
            .query('location_id == @location_id')
            .set_index(['location_id', 'age_group_start', 'sex_id'])
            [['val']]
            .join(
                le_calculated_all_countries_df
                .query('location_id == @location_id')
                .set_index(['location_id', 'age_group_start', 'sex_id'])
                [['E_x_val']],
                how='inner'
            )
        )
        .assign(mae = lambda x: abs(x.E_x_val - x.val))
        [['val', 'E_x_val', 'mae']]
    )

100%|██████████| 204/204 [00:03<00:00, 54.96it/s]


In [76]:
px.histogram(np.array([x.mae.max() for x in le_mean_absolute_error]), nbins=100)

In [77]:
px.line(
    le_mean_absolute_error
    [np.argmax([x.mae.max() for x in le_mean_absolute_error])]
    .reset_index()
    .query('sex_id == 1')
    .set_index('age_group_start')
    .sort_index()
    [['val', 'E_x_val']]
)

In [78]:
le_calculated_all_countries_df

,age_group_start,sex_id,E_x_val,location_id,excluded_rei_id
0,0,1,74.658944,6,NaN
1,1,1,74.207007,6,NaN
2,2,1,73.240265,6,NaN
3,3,1,72.273087,6,NaN
4,4,1,71.305473,6,NaN
...,...,...,...,...,...
203,91,2,3.915640,374,NaN
204,92,2,3.762522,374,NaN
205,93,2,3.571439,374,NaN
206,94,2,3.332979,374,NaN


### Calculate life expectancy extension by risk excluding

In [94]:
len(le_calculated_all_countries_df)

39168

In [98]:
len(le_calculated_risk_excluded_all_countries_df) / le_calculated_risk_excluded_all_countries_df.excluded_rei_id.nunique()

39168.0

In [90]:
(
    le_calculated_risk_excluded_all_countries_df.isna().sum()
    /
    len(le_calculated_risk_excluded_all_countries_df)
)

age_group_start    0.0
sex_id             0.0
E_x_val            0.0
location_id        0.0
excluded_rei_id    0.0
dtype: float64

In [100]:
le_calculated_all_countries_df

,age_group_start,sex_id,E_x_val,location_id,excluded_rei_id
0,0,1,74.658944,6,NaN
1,1,1,74.207007,6,NaN
2,2,1,73.240265,6,NaN
3,3,1,72.273087,6,NaN
4,4,1,71.305473,6,NaN
...,...,...,...,...,...
203,91,2,3.915640,374,NaN
204,92,2,3.762522,374,NaN
205,93,2,3.571439,374,NaN
206,94,2,3.332979,374,NaN


In [106]:
le_calculated_risk_excluded_all_countries_df = (
    le_calculated_risk_excluded_all_countries_df
    .set_index(
        [
            'location_id',
            'age_group_start',
            'sex_id',
        ]
    )
    .join(
        le_calculated_all_countries_df
        .set_index(
            [
                'location_id',
                'age_group_start',
                'sex_id',
            ]
        )
        [['E_x_val']],
        rsuffix='_source',
        how='outer'
    )
    .assign(
        E_x_diff = lambda x: x.E_x_val - x.E_x_val_source
    )
    .reset_index()
)

In [ ]:
le_calculated_risk_excluded_all_countries_df.

In [121]:
life_expectancy_interpolated = data_interpolate_inside_age_groups(
    data=life_expectancy,
    start_age_column='age_group_start',
    columns_to_interpolate=['val'],
    val_columns=['val'],
    columns_for_subsample=['location_id', 'sex_id'],
    columns_for_subsample_is_string=False,
    age_max=111,
    age_max_value=0,
    method='linear',
    divide=False,
)

In [122]:
life_expectancy_interpolated.reset_index(inplace=True)

life_expectancy_interpolated.columns = ['age_group_start',  'val', 'location_id', 'sex_id']
life_expectancy_interpolated = life_expectancy_interpolated[['location_id', 'age_group_start', 'sex_id', 'val']]

In [124]:
life_expectancy_interpolated = life_expectancy_interpolated.query('age_group_start <= 110')

In [138]:
location_id = np.random.choice(life_expectancy.location_id.unique())
sex_id = 1

px.line(
    life_expectancy
    .query('location_id == @location_id and sex_id == @sex_id')
    .set_index(['age_group_start'])
    [['val']]
    .join(
        life_expectancy_interpolated
        .query('location_id == @location_id and sex_id == @sex_id')
        .set_index(['age_group_start'])
        .sort_index(),
        rsuffix='_interpolated',
        how='outer'
    )
    .sort_index()
    [['val', 'val_interpolated']], markers=True
    )

In [139]:
life_expectancy_interpolated

,location_id,age_group_start,sex_id,val
0,8,0,1,77.142649
1,8,1,1,76.445780
2,8,2,1,75.463293
3,8,3,1,74.480806
4,8,4,1,73.498319
...,...,...,...,...
45690,209,106,2,1.848643
45691,209,107,2,1.757339
45692,209,108,2,1.666034
45693,209,109,2,1.574730


In [141]:
le_calculated_risk_excluded_all_countries_df = (
    le_calculated_risk_excluded_all_countries_df
    .set_index(
        [
            'location_id',
            'age_group_start',
            'sex_id',
        ]
    )
    .join(
        life_expectancy_interpolated
        .set_index(
            [
                'location_id',
                'age_group_start',
                'sex_id',
            ]
        )
    )
    .reset_index()
)

In [142]:
le_calculated_risk_excluded_all_countries_df.isna().sum()

location_id        0
age_group_start    0
sex_id             0
excluded_rei_id    0
E_x_val            0
E_x_val_source     0
E_x_diff           0
val                0
dtype: int64

In [144]:
le_calculated_risk_excluded_all_countries_df.columns = [
    'location_id', 'age_group_start', 'sex_id', 'excluded_rei_id',
    'E_x_val_risk_excluded', 'E_x_val_calculated', 'E_x_diff', 'E_x_val'
]

In [145]:
le_calculated_risk_excluded_all_countries_df = le_calculated_risk_excluded_all_countries_df[[
    'location_id', 'age_group_start', 'sex_id', 'excluded_rei_id',
    'E_x_val', 'E_x_val_calculated', 'E_x_val_risk_excluded', 'E_x_diff'
]]

In [178]:
le_calculated_risk_excluded_all_countries_df.columns = [
    'location_id', 'age', 'sex_id', 'rei_id',
    'E_x_val', 'E_x_val_calculated', 'E_x_val_risk_excluded', 'E_x_diff'
]

Saving the results

In [179]:
le_calculated_risk_excluded_all_countries_df.to_csv(
    os.path.join(
        'life_extension_dashboard_app',
        'data',
        'risk_excluded_le.csv'
    ),
    index=False,
)